FiPy scheme for solving phase field crystal model

In [1]:
from fipy import *
import mayavi
import numpy as np


In [2]:
# Create the spherical computational mesh using Gmsh
# Mesh construction from the following example: 
# https://www.ctcms.nist.gov/fipy/examples/cahnHilliard/generated/examples.cahnHilliard.sphere.html

In [3]:
mesh = Gmsh2DIn3DSpace('''
      radius = 15.0;
      cellSize = 0.4;
      
      // create inner 1/8 shell
      Point(1) = {0, 0, 0, cellSize};
      Point(2) = {-radius, 0, 0, cellSize};
      Point(3) = {0, radius, 0, cellSize};
      Point(4) = {0, 0, radius, cellSize};
      Circle(1) = {2, 1, 3};
      Circle(2) = {4, 1, 2};
      Circle(3) = {4, 1, 3};
      Line Loop(1) = {1, -3, 2} ;
      Ruled Surface(1) = {1};
      
      // create remaining 7/8 inner shells
      t1[] = Rotate {{0,0,1},{0,0,0},Pi/2} {Duplicata{Surface{1};}};
      t2[] = Rotate {{0,0,1},{0,0,0},Pi} {Duplicata{Surface{1};}};
      t3[] = Rotate {{0,0,1},{0,0,0},Pi*3/2} {Duplicata{Surface{1};}};
      t4[] = Rotate {{0,1,0},{0,0,0},-Pi/2} {Duplicata{Surface{1};}};
      t5[] = Rotate {{0,0,1},{0,0,0},Pi/2} {Duplicata{Surface{t4[0]};}};
      t6[] = Rotate {{0,0,1},{0,0,0},Pi} {Duplicata{Surface{t4[0]};}};
      t7[] = Rotate {{0,0,1},{0,0,0},Pi*3/2} {Duplicata{Surface{t4[0]};}};
      
      // create entire inner and outer shell
      Surface Loop(100)={1,t1[0],t2[0],t3[0],t7[0],t4[0],t5[0],t6[0]};
  ''', order=2).extrude(extrudeFunc=lambda r: 1.1 * r) 


In [4]:
# Set up the phase field crystal PDE, qn is the characteristic wave number

phi = CellVariable(name=r"$\phi$", mesh=mesh) 
psi = CellVariable(name=r"$\psi$", mesh=mesh) 
psi2 = CellVariable(name=r"$\psi_2$", mesh=mesh) 

# initial field is Gaussian random variable
phi.setValue(GaussianNoiseVariable(mesh=mesh,mean=0.25,variance=0.05)) 
PHI = phi.arithmeticFaceValue 
D = a = epsilon = 1.
qn = 1.0
sourcy = (phi*phi*phi-phi)*1.0+qn**4*phi
eq1 = (TransientTerm(var=phi) == DiffusionTerm(coeff=D, var=psi))
eq2 = (ImplicitSourceTerm(coeff=1., var=psi) == ImplicitSourceTerm(coeff=sourcy,var=phi)+DiffusionTerm(coeff=2*qn**2,var=phi)+DiffusionTerm(coeff=1.,var=psi2))
eq3 = (ImplicitSourceTerm(coeff=1.,var=psi2) == DiffusionTerm(coeff=1.,var=phi))
eq = eq1 & eq2 & eq3

In [5]:
# set up variables for time evolution 
# total elapsed time "elapsed" and exponential time step increase with exponent "dexp"

dexp = -6
elapsed = 0

#evolve until time "duration"

duration = 2
while elapsed < duration:
    dt = min(100, numerix.exp(dexp))
    elapsed += dt
    dexp += 0.1
    eq.solve(dt=dt) 

In [6]:
# convert mesh data to 3d point data


x3d,y3d,z3d=mesh.faceCenters
valc = np.array(phi.arithmeticFaceValue)

rr = -1
r3d=(x3d**2+y3d**2)**0.5
theta3d=np.zeros(len(z3d))
pp3d=np.zeros(len(z3d))
x3dn = np.zeros(len(z3d))
y3dn = np.zeros(len(z3d))
z3dn = np.zeros(len(z3d))


for i in range(0,len(z3d)):
    pp3d[i]=np.arctan2(y3d[i],x3d[i])
    rad = (x3d[i]**2+y3d[i]**2+z3d[i]**2)**0.5
    theta3d[i]=np.arccos(z3d[i]/rad)
    x3dn[i]=(rad+rr*valc[i])*np.cos(pp3d[i])*np.sin(theta3d[i])
    y3dn[i]=(rad+rr*valc[i])*np.sin(pp3d[i])*np.sin(theta3d[i])
    z3dn[i]=(rad+rr*valc[i])*np.cos(theta3d[i])


valc=(valc-valc.min())/(valc.max()-valc.min())

In [7]:
# plot the solution using mayavi

from mayavi import mlab
import matplotlib as mpl
pts = mlab.points3d(x3dn,y3dn,z3dn,scale_factor=0.5)
pts.glyph.scale_mode = 'scale_by_vector'
pts.mlab_source.dataset.point_data.scalars = valc
mlab.show()

In [ ]:
# export solution data to file
from fipy.tools import dump
phi, mesh, psi, psi2, psi3 = dump.read('run1')

In [ ]:
data = dump.read('run1')